In [ ]:
import pandas as pd
import ruptures as rpt
import matplotlib.pyplot as plt
from scipy.signal import find_peaks


vid_1_angles_df = pd.read_csv("../data/pwr/angles/vid_001_angles.csv")

In [ ]:
vid_1_angles_df[["right_knee_pitch", "left_knee_pitch"]].plot()

In [ ]:
import numpy as np

X = np.mean(
    [
        vid_1_angles_df["right_knee_pitch"].to_numpy(),
        vid_1_angles_df["left_knee_pitch"].to_numpy(),
    ],
    axis=0,
)

In [ ]:
plt.plot(X)
plt.xlabel("Klatka")
plt.ylabel("Kąt")
plt.title("Sygnał ćwiczenia")

## ChangePoints

In [ ]:
algo = rpt.Pelt(model="rbf").fit(X)
result = algo.predict(pen=10)

# Wyświetlanie wyników
print("Punkty zmiany:", result)

In [ ]:
fig, ax = plt.subplots()

ax.plot(X)

for resul in result:
    ax.axvline(x=resul, color="b", ls="--")

## Peaks

In [ ]:
mean_signal_value = X.mean()
peaks, peaks_info = find_peaks(X, height=mean_signal_value, distance=15)
valleys, _ = find_peaks(-X, height=-mean_signal_value, distance=15)

In [ ]:
fig, ax = plt.subplots()

ax.plot(X)

# for peak in peaks:
ax.plot(peaks, X[peaks], "x")
ax.plot(valleys, X[valleys], "o")
ax.set_xlabel("Klatka")
ax.set_ylabel("Kąt")
ax.set_title("Wykrycie wierzchołków")

In [ ]:
valleys

In [ ]:
peaks = [6, 95, 189, 278, 360, 449]
valleys = [64, 164, 248, 334, 419]

In [ ]:
fig, ax = plt.subplots()

ax.plot(X)

# for peak in peaks:
ax.plot(peaks, X[peaks], "x")
ax.plot(valleys, X[valleys], "o")
ax.set_xlabel("Klatka")
ax.set_ylabel("Kąt")
ax.set_title("Przefiltrowane wierzchołki")

In [ ]:
def get_segments(peaks: list, valleys: list) -> list[list[int, int]]:
    segments = []
    valley_idx = 0
    peaks_idx = 0
    while peaks_idx < len(peaks) - 1:
        if peaks[peaks_idx] < valleys[valley_idx]:
            if peaks[peaks_idx + 1] < valleys[valley_idx]:
                peaks[peaks_idx + 1] = (peaks[peaks_idx] + peaks[peaks_idx + 1]) // 2
            else:
                segments.append([peaks[peaks_idx], peaks[peaks_idx + 1]])
            peaks_idx += 1

        else:
            if valley_idx >= len(valleys) - 1:
                break
            if valleys[valley_idx + 1] < peaks[peaks_idx]:
                valleys[valley_idx + 1] = (
                    valleys[valley_idx] + valleys[valley_idx + 1]
                ) // 2
            valley_idx += 1
    return segments

In [ ]:
segments = get_segments(peaks, valleys)